In [4]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# Paths
input_path = r"C:\Users\black\Downloads\microdatos_manzana\Microdatos_Manzana.shp"
out_dir = Path(r"C:\Users\black\Downloads\microdatos_manzana\Centroide")
out_dir.mkdir(parents=True, exist_ok=True)

# Load
gdf = gpd.read_file(input_path)

# Regiones para EPSG 32718
regiones_epsg32718 = [
    "REGIÓN DE ATACAMA",
    "REGIÓN DE ARICA Y PARINACOTA",
    "REGIÓN DE COQUIMBO",
    "REGIÓN DE VALPARAÍSO",
    "REGIÓN DE TARAPACÁ",
    "REGIÓN DE ANTOFAGASTA"
]

# Calcular centroides
gdf["geometry"] = gdf.geometry.centroid

# Dividir y reproyectar
gdf_18 = gdf[gdf["REGION"].isin(regiones_epsg32718)].to_crs(epsg=32718)[["CUT", "TOTAL_PERS", "geometry"]]
gdf_19 = gdf[~gdf["REGION"].isin(regiones_epsg32718)].to_crs(epsg=32719)[["CUT", "TOTAL_PERS", "geometry"]]

# Guardar shapefiles separados
out18 = out_dir / "Centroides_Manzana_32718.shp"
out19 = out_dir / "Centroides_Manzana_32719.shp"

gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")
gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")

print(f"Exportado: {out18}")
print(f"Exportado: {out19}")



Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Centroides_Manzana_32718.shp
Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Centroides_Manzana_32719.shp


In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path

# === Paths ===
poly_path = r"C:\Users\black\Downloads\microdatos_entidad\Microdatos_Entidad.shp"
points_path = r"C:\Users\black\Downloads\viviendas-rurales-precenso-2016\Viviendas_Rurales_Precenso_2016.shp"
out_dir = Path(r"C:\Users\black\Downloads\microdatos_manzana\Centroide")
out_dir.mkdir(parents=True, exist_ok=True)

# === Leer shapefiles ===
polys = gpd.read_file(poly_path)
pts = gpd.read_file(points_path)

# === Regiones EPSG 32718 ===
regiones_32718 = [1, 2, 3, 4, 5, 15]

# Asegurar que ambos tengan el mismo CRS para hacer spatial join
if polys.crs != pts.crs:
    pts = pts.to_crs(polys.crs)

# === Join espacial: asignar puntos a polígonos ===
joined = gpd.sjoin(pts, polys, how="inner", predicate="within")

# === Calcular promedio de coordenadas por polígono ===
means = (
    joined.groupby("index_right")
    .apply(lambda g: pd.Series({
        "x_mean": g.geometry.x.mean(),
        "y_mean": g.geometry.y.mean()
    }))
)

# === Unir resultados con atributos de los polígonos ===
polys_reset = polys.reset_index()
result = polys_reset.merge(means, left_index=True, right_index=True)

# Crear geometría con el promedio
result["geometry"] = [Point(xy) for xy in zip(result["x_mean"], result["y_mean"])]

# Seleccionar columnas necesarias
result = result[["COD_COMUNA", "TOTAL_PERS", "COD_REGION", "geometry"]]
result = gpd.GeoDataFrame(result, geometry="geometry", crs=polys.crs)

# === Separar por EPSG ===
gdf_18 = result[result["COD_REGION"].astype(int).isin(regiones_32718)].to_crs(epsg=32718)
gdf_19 = result[~result["COD_REGION"].astype(int).isin(regiones_32718)].to_crs(epsg=32719)

# === Guardar ===
out18 = out_dir / "Entidad_Promedios_32718.shp"
out19 = out_dir / "Entidad_Promedios_32719.shp"

gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")
gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")

print(f"Exportado: {out18}")
print(f"Exportado: {out19}")


C:\Users\black\AppData\Local\Temp\ipykernel_24588\1824467658.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Entidad_Promedios_32718.shp
Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Entidad_Promedios_32719.shp


In [12]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path

# === Paths ===
excel_path = r"C:\Users\black\Documents\SINCA\stations_info_stata.xlsx"
out_dir = Path(r"C:\Users\black\Downloads\microdatos_manzana\Centroide")
out_dir.mkdir(parents=True, exist_ok=True)

# === Leer Excel ===
df = pd.read_excel(excel_path)

# Eliminar duplicados por Estación
df = df.drop_duplicates(subset=["Estación"])

# Crear geometría desde Lat / Lon (en WGS84)
geometry = [Point(xy) for xy in zip(df["Longitud"], df["Latitud"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Regiones que van con EPSG 32718
regiones_32718 = ["I", "II", "III", "IV", "V", "XV"]

# Dividir según región y transformar CRS
gdf_18 = gdf[gdf["Regióncódigo"].isin(regiones_32718)].to_crs(epsg=32718)
gdf_19 = gdf[~gdf["Regióncódigo"].isin(regiones_32718)].to_crs(epsg=32719)

# Convertir fechas a string (si existen)
for col in ["date_inicio", "date_fin"]:
    if col in gdf_18.columns:
        gdf_18[col] = gdf_18[col].astype(str)
    if col in gdf_19.columns:
        gdf_19[col] = gdf_19[col].astype(str)

# === Exportar ===
out18 = out_dir / "Coordenadas_LatLon_32718.shp"
out19 = out_dir / "Coordenadas_LatLon_32719.shp"

if not gdf_18.empty:
    gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")
    print(f"Exportado: {out18}")

if not gdf_19.empty:
    gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")
    print(f"Exportado: {out19}")


Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp
Exportado: C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp


C:\Users\black\AppData\Local\Temp\ipykernel_22568\1534637724.py:40: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_18.to_file(out18, driver="ESRI Shapefile", encoding="utf-8")
C:\Users\black\AppData\Local\Temp\ipykernel_22568\1534637724.py:44: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_19.to_file(out19, driver="ESRI Shapefile", encoding="utf-8")


In [4]:
import geopandas as gpd
import pandas as pd
from shapely.ops import nearest_points
import numpy as np


def build_df(entidades_path, centros_path, csv_path, radius_km=2):
    # Load shapefiles
    entidades = gpd.read_file(entidades_path)
    centros = gpd.read_file(centros_path)

    # Load allowed Estaciones from CSV
    valid_estaciones = pd.read_csv(csv_path)["Estación"].astype(str).unique()

    # Keep only matching centros
    centros = centros[centros["Estación"].astype(str).isin(valid_estaciones)].copy()

    # Project to metric CRS
    entidades = entidades.to_crs(epsg=32719)  # Adjust CRS if needed
    centros = centros.to_crs(entidades.crs)

    records = []

    for idx, ent in entidades.iterrows():
        ent_geom = ent.geometry

        # Compute distances
        centros["dist_m"] = centros.geometry.distance(ent_geom)
        nearby = centros[centros["dist_m"] <= radius_km * 1000].copy()

        if nearby.empty:
            records.append({
                "Entidad": ent.get("name", idx),  # adjust col name
                "Estación": ".",
                "cod_comuna": ent["CUT"],
                "weight": 0
            })
        else:
            # Inverse distance weights
            nearby["inv_dist"] = 1 / nearby["dist_m"]
            nearby["weight"] = nearby["inv_dist"] / nearby["inv_dist"].sum()

            for _, cen in nearby.iterrows():
                records.append({
                    "Entidad": ent.get("name", idx),
                    "Estación": cen["Estación"],
                    "cod_comuna": ent["CUT"],  # keep original name
                    "weight": cen["weight"]
                })

    return pd.DataFrame(records)

def build_df2(entidades_path, centros_path, csv_path, radius_km=2):
    # Load shapefiles
    entidades = gpd.read_file(entidades_path)
    centros = gpd.read_file(centros_path)

    # Load allowed Estaciones from CSV
    valid_estaciones = pd.read_csv(csv_path)["Estación"].astype(str).unique()

    # Keep only matching centros
    centros = centros[centros["Estación"].astype(str).isin(valid_estaciones)].copy()

    # Project to metric CRS
    entidades = entidades.to_crs(epsg=32719)  # Adjust CRS if needed
    centros = centros.to_crs(entidades.crs)

    records = []

    for idx, ent in entidades.iterrows():
        ent_geom = ent.geometry

        # Compute distances
        centros["dist_m"] = centros.geometry.distance(ent_geom)
        nearby = centros[centros["dist_m"] <= radius_km * 1000].copy()

        if nearby.empty:
            records.append({
                "Entidad": ent.get("name", idx),  # adjust col name
                "Estación": ".",
                "cod_comuna": ent["COD_COMUNA"],
                "weight": 0
            })
        else:
            # Inverse distance weights
            nearby["inv_dist"] = 1 / nearby["dist_m"]
            nearby["weight"] = nearby["inv_dist"] / nearby["inv_dist"].sum()

            for _, cen in nearby.iterrows():
                records.append({
                    "Entidad": ent.get("name", idx),
                    "Estación": cen["Estación"],
                    "cod_comuna": ent["COD_COMUNA"],  # keep original name
                    "weight": cen["weight"]
                })

    return pd.DataFrame(records)

# ---- USAGE ----
df1 = build_df(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_32718.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=2)
df2 = build_df(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_32719.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=2)
df3 = build_df2(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_32718.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=2)
df4 = build_df2(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_32719.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=2)
# Append both results
df1["shp"] = 1
df2["shp"] = 2
df3["shp"] = 3
df4["shp"] = 4
final_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Export to Excel
final_df.to_excel(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\entidades_centros.xlsx", index=False)
# ---- USAGE ----
df21 = build_df(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_32718.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=5)
df22 = build_df(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_32719.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=5)
df23 = build_df2(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_32718.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=5)
df24 = build_df2(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_32719.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=5)
# Append both results
df21["shp"] = 1
df22["shp"] = 2
df23["shp"] = 3
df24["shp"] = 4
final_df2 = pd.concat([df21, df22, df23, df24], ignore_index=True)

# Export to Excel
final_df2.to_excel(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\entidades_centros2.xlsx", index=False)
# ---- USAGE ----
df31 = build_df(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_32718.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=10)
df32 = build_df(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Centroides_Manzana_32719.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=10)
df33 = build_df2(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_32718.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32718.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=10)
df34 = build_df2(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\Entidad_Promedios_32719.shp", r"C:\Users\black\Downloads\microdatos_manzana\Centroide\Coordenadas_LatLon_32719.shp", r"C:\Users\black\Documents\SINCA\centros.csv", radius_km=10)
# Append both results
df31["shp"] = 1
df32["shp"] = 2
df33["shp"] = 3
df34["shp"] = 4
final_df3 = pd.concat([df31, df32, df33, df34], ignore_index=True)

# Export to Excel
final_df3.to_excel(r"C:\Users\black\Dropbox\Proyectos\microdatos_manzana\Centroide\entidades_centros3.xlsx", index=False)
